In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from openquake.hazardlib.gsim import get_available_gsims
from openquake.hazardlib.gsim.base import RuptureContext, SitesContext, DistancesContext
from openquake.hazardlib.imt import PGA, SA, PGV
from openquake.hazardlib import const
from openquake.hazardlib.geo import (Point, Line, Polygon, Mesh,
                                     SimpleFaultSurface,
                                     PlanarSurface,
                                     ComplexFaultSurface)

### Running GMPEs (and plotting them) with OpenQuake

##### Selecting the GMPE

The complete documentation for OpenQuake GMPEs (updated nightly) can be found here:

https://docs.openquake.org/oq-engine/master/openquake.hazardlib.gsim.html

A quick command to show the names of all the GMPEs available in the version you are using can be seen below

In [ ]:
gmpe_list = get_available_gsims()
for gmpe_name in gmpe_list:
    print(gmpe_name)

<b>What information does your GMPE require in order to calculate the ground motion?</b>

OpenQuake uses objects ("Contexts") to organise this information into Rupture parameters, Site parameters and Distance (Path) parameters. Every GMPE implemented in OpenQuake specifies which parameters is needs.

For example, let's take a look at Boore et al. (2014)

In [ ]:
boore_2014 = gmpe_list["BooreEtAl2014"]()
print(boore_2014.REQUIRES_RUPTURE_PARAMETERS)
print(boore_2014.REQUIRES_DISTANCES)
print(boore_2014.REQUIRES_SITES_PARAMETERS)

So, let's set up a simple scenario for this GMPE

I want to see the "scenario spectrum" from this GMPE for a site located at a distance of 20 km from the surface projection of the fault rupture (the "Joyner-Boore" distance, or $R_{JB}$). The magnitude of the earthquake is $M_W$ 6.5 and the style of faulting is reverse.

OpenQuake uses rake to define style-of-faulting according to the Aki & Richards (2002) convention:

Normal faulting: Rake = -90$^{\circ}$

Reverse faulting: Rake = 90$^{\circ}$

Strike-slip faulting: Rake = 0$^{\circ}$ or 180$^{\circ}$ 

Finally, we will assume that the site has an averaged 30-m shearwave velocity ($V_{S30}$) of 570 m/s



##### Setup the configuration

In [ ]:
# Rupture Context
rctx = RuptureContext()
rctx.mag = 6.5
rctx.rake = 90.0

# Sites Context - Note this must be input as a numpy array
sctx = SitesContext()
sctx.vs30 = np.array([570.])

# Distances Context - This must also be input as an array
dctx = DistancesContext()
dctx.rjb = np.array([20.0])

Now choose the period range for which we which to calculate ground motion. The selected GMPE Boore et al. ranges from 0.01 s to 10.0 s

In [ ]:
# A quick trick - define 70 values logarithmically spaced between 0.01 and 10
periods = np.logspace(-2., 1., 70)
print(periods)

Need to turn these into "intensity measure type (IMT)" objects

In [ ]:
imts = [SA(per) for per in periods]
print(imts)

Some notes: 

OpenQuake's GMPEs are vectorised by sites/distances (not periods)!

The GMPEs also return their standard deviations - can take this into account

The GMPEs return the natural logarithm of the mean ground motion

In [ ]:
# Asking the GMPE to return only the total standard deviation
stddev_types = [const.StdDev.TOTAL]

Now lets build the scenario spectum (need to loop over periods)

In [ ]:
# Pre-allocate results vectors
means = np.zeros_like(periods)
means_plus_1sd = np.zeros_like(periods)
means_minus_1sd = np.zeros_like(periods)
stddevs = np.zeros_like(periods)

# Loop over each IMT
for i, imt in enumerate(imts):
    # Call the function `get_mean_and_stddevs` - which every GMPE has
    mean, [stddev] = boore_2014.get_mean_and_stddevs(sctx, rctx, dctx, imt, stddev_types)
    means[i] = np.exp(mean)
    means_plus_1sd[i] = np.exp(mean + stddev)
    means_minus_1sd[i] = np.exp(mean - stddev)
    stddevs[i] = stddev

# View the results
print("Period (s)  Sa (g)   Sigma")
for imt, mean, stddev in zip(imts, means, stddevs):
    print("{:10.4f}  {:6.4f}  {:6.4f}".format(imt.period, mean, stddev))

Now plot the mean ground motions plus/minus one standard deviation

In [ ]:
plt.figure(figsize=(8,8))
plt.semilogx(periods, means, "b-", lw=2, label="Mean")
plt.semilogx(periods, means_plus_1sd, "b--", lw=1.2)
plt.semilogx(periods, means_minus_1sd, "b--", lw=1.2, label=r"$\pm 1 \sigma$")
# Add grids, labels, set limits etc
plt.grid()
plt.xlabel("Period", fontsize=16)
plt.ylabel("Sa (g)", fontsize=16)
plt.xlim(0.01, 10.)
plt.ylim(bottom=0.0)
plt.legend(fontsize=18)

### Now let's compare several GMPEs

So far we have plotted the scenario spectrum for a single GMPEs. We might be interested to know how several GMPEs compare.

Let's compare one "Global" model (Boore et al. 2014), one "European" Model (Akkar et al., 2014) and one model based mostly on Japanese data (Cauzzi et al. 2014)

Firstly, what information do they need:

In [ ]:
# Define the GMPE list
gmpes = [gmpe_list["BooreEtAl2014"](), gmpe_list["AkkarEtAlRjb2014"](), gmpe_list['CauzziEtAl2014']()]

for gmpe in gmpes:
    # Each GMPE has the following information
    print(str(gmpe))
    print(gmpe.REQUIRES_RUPTURE_PARAMETERS)
    print(gmpe.REQUIRES_DISTANCES)
    print(gmpe.REQUIRES_SITES_PARAMETERS)

It seems that Cauzzi et al. is using a different distance metric - so need to define it.

Let's imagine our rupture doesn't reach the surface - it stops about 2 km below it (is a "blind fault").

For convenience we will also assume that our site is on the "footwall" of the rupture (i.e. it is dipping away from the site)

In [ ]:
dctx.rrup = np.sqrt(dctx.rjb ** 2. + 2.0 ** 2.)

Now to run the GMPEs. It is helpful to keep the results organised, so I will use a dictionary for this

In [ ]:
results = {} # Empty dictionary
for gmpe in gmpes:
    print("Running GMPE %s" % str(gmpe))
    # Add a results dictionary for each GMPE
    results[str(gmpe)] = {"mean": np.zeros_like(periods),
                          "stddevs": np.zeros_like(periods),
                          "mean_plus_1sd": np.zeros_like(periods),
                          "mean_minus_1sd": np.zeros_like(periods)}
    for i, imt in enumerate(imts):
        mean, [stddev] = gmpe.get_mean_and_stddevs(sctx, rctx, dctx, imt, stddev_types)
        results[gmpe]["mean"][i] = np.exp(mean)
        results[gmpe]["stddevs"][i] = stddev
        results[gmpe]["mean_plus_1sd"][i] = np.exp(mean + stddev)
        results[gmpe]["mean_minus_1sd"][i] = np.exp(mean - stddev)

<b>What happened?</b>

Not all of the GMPEs are defined for the same period range - Akkar et al. (2014) only goes up to 4 seconds.

We could just limit our spectrum to the periods that are common to all GMPEs. Easy but then you might miss interesting results!

Instead, lets use Python's "try-catch" functionality to get values when they can be calculated and just return "not-a-number" (`np.nan`) otherwise

In [ ]:
results = {} # Empty dictionary
for gmpe in gmpes:
    print("Running GMPE %s" % str(gmpe))
    results[str(gmpe)] = {"mean": np.zeros_like(periods),
                          "stddevs": np.zeros_like(periods),
                          "mean_plus_1sd": np.zeros_like(periods),
                          "mean_minus_1sd": np.zeros_like(periods)}
    for i, imt in enumerate(imts):
        try:
            mean, [stddev] = gmpe.get_mean_and_stddevs(sctx, rctx, dctx, imt, stddev_types)
            results[gmpe]["mean"][i] = np.exp(mean)
            results[gmpe]["stddevs"][i] = stddev
            results[gmpe]["mean_plus_1sd"][i] = np.exp(mean + stddev)
            results[gmpe]["mean_minus_1sd"][i] = np.exp(mean - stddev)
        except KeyError:
            # If it raises the error we have just seen (a KeyError) - put in nans
            results[gmpe]["mean"][i] = np.nan
            results[gmpe]["stddevs"][i] = np.nan
            results[gmpe]["mean_plus_1sd"][i] = np.nan
            results[gmpe]["mean_minus_1sd"][i] = np.nan

We will find we need to use this last piece of code over again - so let's turn it into a function

In [ ]:
def calculate_ground_motions(gmpes, imts, sctx, rctx, dctx, stddev_types):
    """
    It's good practice to comment functions ... so here it is:

    Calculates the expected ground motion and uncertainty, organised by GMPE
    and intensity measure type (i.e. PGA, SA etc.), for a given rupture-site configuration    
    """
    results = {} # Empty dictionary
    nper = len(imts)
    for gmpe in gmpes:
        print("Running GMPE %s" % str(gmpe))
        results[str(gmpe)] = {"mean": np.zeros(nper),
                              "stddevs": np.zeros(nper),
                              "mean_plus_1sd": np.zeros(nper),
                              "mean_minus_1sd": np.zeros(nper)}
        for i, imt in enumerate(imts):
            try:
                mean, [stddev] = gmpe.get_mean_and_stddevs(sctx, rctx, dctx, imt, stddev_types)
                results[gmpe]["mean"][i] = np.exp(mean)
                results[gmpe]["stddevs"][i] = stddev
                results[gmpe]["mean_plus_1sd"][i] = np.exp(mean + stddev)
                results[gmpe]["mean_minus_1sd"][i] = np.exp(mean - stddev)
            except KeyError:
                # If it raises the error we have just seen (a KeyError) - put in nans
                results[gmpe]["mean"][i] = np.nan
                results[gmpe]["stddevs"][i] = np.nan
                results[gmpe]["mean_plus_1sd"][i] = np.nan
                results[gmpe]["mean_minus_1sd"][i] = np.nan
    return results

Plot the scenario results

In [ ]:
plt.figure(figsize=(8,8))
# The zip function joins together two lists - here we are linking each list with a specific plotting colour
for gmpe, color in zip(gmpes, ["r", "b", "k"]):
    # Plot the mean and plus/minus 1 stddev for each GMPE
    plt.semilogx(periods, results[str(gmpe)]["mean"], "-", color=color, lw=2, label=str(gmpe))
    plt.semilogx(periods, results[str(gmpe)]["mean_plus_1sd"], "--", color=color, lw=1.2)
    plt.semilogx(periods, results[str(gmpe)]["mean_minus_1sd"], "--", color=color, lw=1.2)
plt.grid()
plt.xlabel("Period", fontsize=16)
plt.ylabel("Sa (g)", fontsize=16)
plt.xlim(0.01, 10.)
plt.ylim(bottom=0.0)
plt.legend(fontsize=16)

We can also take a look at the total standard deviations too

In [ ]:
plt.figure(figsize=(8,8))
for gmpe, color in zip(gmpes, ["r", "b", "k"]):
    plt.semilogx(periods, results[str(gmpe)]["stddevs"], "-", color=color, lw=2, label=str(gmpe))

plt.grid()
plt.xlabel("Period", fontsize=16)
plt.ylabel("Total Std. Dev", fontsize=16)
plt.xlim(0.01, 10.)
plt.ylim(0.0, 1.0)
plt.legend(fontsize=16)

### Comparing GMPEs - Attenuation with distance

We have seen how the spectra of the GMPEs compare for a single site and single scenario - clearly there are differences?

To understand more perhaps we can compare how the ground motion is changing with distance from the rupture

In [ ]:
dctx = DistancesContext()
# Now define a vector of distances from 0 km to 150 km
dctx.rjb = np.arange(0., 151., 1.)
dctx.rrup = np.sqrt(dctx.rjb ** 2. + 2.0 ** 2.)

# Now we have a vector of distances we need a vector of site terms
sctx = SitesContext()
sctx.vs30 = 570.0 * np.ones_like(dctx.rjb)

Let's take a look at two intensity measures: PGA and 0.3 s spectral acceleration

In [ ]:
nsites = len(dctx.rjb)
# Setup an empty dictionary to store results
results = {}
# Define our two intensity measures
imts = [PGA(), SA(0.3)]
for imt in imts:
    imt_name = str(imt)
    # For each intensity measure - create it's own empty dictionary
    results[imt_name] = {}
    print(imt_name)
    for gmpe in gmpes:
        gmpe_name = str(gmpe)
        print(gmpe_name)
        results[imt_name][gmpe_name] = {}
        # Run the GMPE
        mean, [stddev] = gmpe.get_mean_and_stddevs(sctx, rctx, dctx, imt, stddev_types)
        # Organise the results
        results[imt_name][gmpe_name]["mean"] = np.exp(mean)
        results[imt_name][gmpe_name]["stddev"] = stddev
        results[imt_name][gmpe_name]["mean_plus_1sd"] = np.exp(mean + stddev)
        results[imt_name][gmpe_name]["mean_minus_1sd"] = np.exp(mean - stddev)

Now plot the results

In [ ]:
fig = plt.figure(figsize=(12, 7))
# Using subplots here
# The first subplot shows the results for PGA
ax1 = fig.add_subplot(121)
for gmpe, color in zip(gmpes, ["r", "b", "k"]):
    ax1.loglog(dctx.rjb, results["PGA"][str(gmpe)]["mean"], "-", color=color, lw=2, label=str(gmpe))
    ax1.loglog(dctx.rjb, results["PGA"][str(gmpe)]["mean_plus_1sd"], "--", color=color, lw=1.2)
    ax1.loglog(dctx.rjb, results["PGA"][str(gmpe)]["mean_minus_1sd"], "--", color=color, lw=1.2)
ax1.set_xlim(1., 150.)
ax1.set_ylim(0.005, 3.)
ax1.set_xlabel(r"Distance $R_{JB}$ (km)", fontsize=14)
ax1.set_ylabel("PGA (g)", fontsize=14)
ax1.grid(True)

# The second subplot shows the results for Sa(0.3 s)
ax2 = fig.add_subplot(122)
for gmpe, color in zip(gmpes, ["r", "b", "k"]):
    ax2.loglog(dctx.rjb, results["SA(0.3)"][str(gmpe)]["mean"], "-", color=color, lw=2, label=str(gmpe))
    ax2.loglog(dctx.rjb, results["SA(0.3)"][str(gmpe)]["mean_plus_1sd"], "--", color=color, lw=1.2)
    ax2.loglog(dctx.rjb, results["SA(0.3)"][str(gmpe)]["mean_minus_1sd"], "--", color=color, lw=1.2)
ax2.set_xlim(1., 150.)
ax2.set_ylim(0.005, 3.)
ax2.set_xlabel(r"Distance $R_{JB}$ (km)", fontsize=14)
ax2.set_ylabel("SA(0.3 s) (g)", fontsize=14)
ax2.grid(True)
ax2.legend(fontsize=14)

In [ ]:
fig = plt.figure(figsize=(12, 7))
# Using subplots here
# The first subplot shows the results for PGA
ax1 = fig.add_subplot(121)
for gmpe, color in zip(gmpes, ["r", "b", "k"]):
    ax1.semilogx(dctx.rjb, results["PGA"][str(gmpe)]["stddev"], "-", color=color, lw=2, label=str(gmpe))
ax1.set_xlim(1., 150.)
ax1.set_ylim(0.0, 1.)
ax1.set_xlabel(r"Distance $R_{JB}$ (km)", fontsize=14)
ax1.set_ylabel("Total Standard Deviation, PGA", fontsize=14)
ax1.grid(True)

# The second subplot shows the results for Sa(0.3 s)
ax2 = fig.add_subplot(122)
for gmpe, color in zip(gmpes, ["r", "b", "k"]):
    ax2.semilogx(dctx.rjb, results["SA(0.3)"][str(gmpe)]["stddev"], "-", color=color, lw=2, label=str(gmpe))
ax2.set_xlim(1., 150.)
ax2.set_ylim(0.0, 1)
ax2.set_xlabel(r"Distance $R_{JB}$ (km)", fontsize=14)
ax2.set_ylabel("Total Standard Deviation, SA(0.3 s)", fontsize=14)
ax2.grid(True)
ax2.legend(fontsize=14)

# How to use OpenQuake to setup a rupture and site configuration

We have seen how to use OpenQuake's GMPE library to plot GMPEs, but the rupture configurations have been quite simplified.
 
In the following example we will take a "real" scenario earthquake (Basel) and calculate the expected ground motions for two cities: Basel and Freiburg

<b>Basel</b> = 7.6$^{\circ}$E, 47.567$^{\circ}$N   $V_{S30}$ = 200 m/s

<b>Freiburg</b> = 7.85$^{\circ}$E, 47.983$^{\circ}$N  $V_{S30}$ = 500 m/s 

The Basel earthquake scenario we are considering has the following properties:

1. $M_W = 6.5$

2. Reverse faulting (rake = $90^{\circ}$)

3. Top of rupture depth of 3 km

4. E-W rupture, dipping north at $40^{\circ}$

5. Aspect ratio (L/W) of 1.0

6. Wells & Coppersmith Magnitude Scaling relation

In [ ]:
# Import the scaling relation
from openquake.hazardlib.scalerel import WC1994

In [ ]:
# Set up the rupture information
mag = 6.5
rake = 90.0
aspect = 1.0
msr = WC1994()
# Get the area, length and width
area = msr.get_median_area(mag, rake)
length = np.sqrt(area * aspect)
width = area / length
print("Area = %.3f km^2  Length = %.3f km   Width = %.3f km" % (area, length, width))

We will assume the top edge of the rupture is described by an east-west trending line with length as we have just calculated and a westerly extent of 7.533$^{\circ}$E, 47.577$^{\circ}$N

In [ ]:
p1 = Point(7.533, 47.577, 3.)

# OpenQuake's Point object has a useful function .point_at
# This tells you the location (on the earth) of a point located at a given
# along-surface distance, vertical distance and azimuth
p2 = p1.point_at(length, # along the surface distance 
                 0., # Vertical distance
                 90.) # Azimuth w.r.t. North (so east-west here)
print("Top of Rupture\n%s to \n%s" % (str(p1), str(p2)))

Define the bottom two corners

In [ ]:
dip = 40.0

# Need the along surface width of the fault
surface_width = width * np.cos(np.radians(dip))
# Need the vertical depth range of the fault
depth_range = width * np.sin(np.radians(dip))

# Bottom two corners - north of the top two
p3 = p1.point_at(surface_width, depth_range, 0.0)
p4 = p2.point_at(surface_width, depth_range, 0.0)

print("Bottom of Rupture\n%s to \n%s" % (str(p3), str(p4)))

Now build an OpenQuake planar surface

In [ ]:
surface = PlanarSurface.from_corner_points(1.0,
                                           top_left=p2, top_right=p1,
                                           bottom_left=p4, bottom_right=p3)
surface.get_mesh()

Define our two target locations

In [ ]:
basel = Point(7.6, 47.567, 0.0)
freiburg = Point(7.85, 47.983, 0.0)

Now lets plot our configuration in 3D

In [ ]:
from mpl_toolkits.mplot3d import Axes3D

In [ ]:
fig = plt.figure(figsize=(8,8))
# For 3D plotting we add a new projection keyword to the subplot
ax = fig.add_subplot(111, projection="3d")
# Plot the fault surface as a wireframce
ax.plot_wireframe(surface.mesh.lons, surface.mesh.lats, -surface.mesh.depths, color="b")
# Plot the two sites
ax.scatter([basel.longitude], [basel.latitude], [basel.depth], s=40, color="g", marker="o")
ax.scatter([freiburg.longitude], [freiburg.latitude], [freiburg.depth], s=40, color="r", marker="s")

### Basel: Ground Motion

With the rupture and site figured we can set up the GMPE calculations

In [ ]:
# Rupture information
rctx = RuptureContext()
rctx.mag = mag
rctx.rake = rake
rctx.ztor = 3.0  # Top of rupture depth - not used for these GMPEs

# Basel has 200 m/s Vs30
sctx = SitesContext()
sctx.vs30 = np.array([200.])

dctx = DistancesContext()
# Calculate the Joyner-Boore distance
dctx.rjb = surface.get_joyner_boore_distance(Mesh.from_points_list([basel]))
# Calculate the shortests distance to the rupture (rupture distance)
dctx.rrup = surface.get_min_distance(Mesh.from_points_list([basel]))
print(dctx.rjb, dctx.rrup)

Now get the scenario spectrum for Basel

In [ ]:
imts = [SA(per) for per in periods]

# See - I said we would need this bit of code again! Here is the function we defined earlier
results = calculate_ground_motions(gmpes, imts, sctx, rctx, dctx, stddev_types)

# Plot the results
plt.figure(figsize=(8,8))
for gmpe, color in zip(gmpes, ["r", "b", "k"]):
    plt.semilogx(periods, results[str(gmpe)]["mean"], "-", color=color, lw=2, label=str(gmpe))
    plt.semilogx(periods, results[str(gmpe)]["mean_plus_1sd"], "--", color=color, lw=1.2)
    plt.semilogx(periods, results[str(gmpe)]["mean_minus_1sd"], "--", color=color, lw=1.2)
plt.grid()
plt.xlabel("Period", fontsize=16)
plt.ylabel("Sa (g)", fontsize=16)
plt.xlim(0.01, 10.)
plt.ylim(bottom=0.0)
plt.legend(fontsize=16)

In [ ]:
# And plot the total standard deviations
plt.figure(figsize=(8,8))
for gmpe, color in zip(gmpes, ["r", "b", "k"]):
    plt.semilogx(periods, results[str(gmpe)]["stddevs"], "-", color=color, lw=2, label=str(gmpe))

plt.grid()
plt.xlabel("Period", fontsize=16)
plt.ylabel("Total Std. Dev", fontsize=16)
plt.xlim(0.01, 10.)
plt.ylim(0.0, 1.0)
plt.legend(fontsize=16)

### Freiburg Ground Motion

As with Basel, now do the same for Freiburg

In [ ]:
# New distances
dctx = DistancesContext()
dctx.rjb = surface.get_joyner_boore_distance(Mesh.from_points_list([freiburg]))
dctx.rrup = surface.get_min_distance(Mesh.from_points_list([freiburg]))
print(dctx.rjb, dctx.rrup)

# New site Vs30 values
sctx = SitesContext()
sctx.vs30 = np.array([500.])

In [ ]:
results = calculate_ground_motions(gmpes, imts, sctx, rctx, dctx, stddev_types)            

plt.figure(figsize=(8,8))
for gmpe, color in zip(gmpes, ["r", "b", "k"]):
    plt.semilogx(periods, results[str(gmpe)]["mean"], "-", color=color, lw=2, label=str(gmpe))
    plt.semilogx(periods, results[str(gmpe)]["mean_plus_1sd"], "--", color=color, lw=1.2)
    plt.semilogx(periods, results[str(gmpe)]["mean_minus_1sd"], "--", color=color, lw=1.2)
plt.grid()
plt.xlabel("Period", fontsize=16)
plt.ylabel("Sa (g)", fontsize=16)
plt.xlim(0.01, 10.)
plt.ylim(bottom=0.0)
plt.legend(fontsize=16)

# Bonus Challenge: What is the probability that the Basel earthquake will damage my structure?

An engineer tells you ...

"If my structure is subject to 0.15 g PGA it will suffer a little damage (e.g. cracks)"

"If my structure is subject to 0.3 g PGA it will suffer moderate damage"

"If my structure is subject to 0.5 g PGA it will suffer extensive damage, but it should not collapse"

"If my structure is subject to 0.8 g PGA it will collapse!"

If this Basel earthquake occurs what is the probability that the structure will suffer from little, moderate, and extensive damage? 

What is the probability it will collapse?

The structure is located in Basel!